# LLM Chain

1. 大模型之chain使用

In [1]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.chains.summarize import load_summarize_chain
from langchain.chains.question_answering import load_qa_chain

import warnings
warnings.filterwarnings('ignore')

### Load Documents

In [3]:
lg_loader = UnstructuredFileLoader("../data/小红帽.txt")
lg_doc = lg_loader.load()

In [4]:
def doc_summary(docs):
    print (f'You have {len(docs)} document(s)')
    num_words = sum([len(doc.page_content.split(' ')) for doc in docs])
    
    print (f'You have roughly {num_words} words in your docs')
    print (f'Preview: \n{docs[0].page_content.split(". ")[0]}')

In [5]:
doc_summary(lg_doc)

You have 1 document(s)
You have roughly 1 words in your docs
Preview: 
从前有个可爱的小姑娘,谁见了都喜欢,但最喜欢她的是她的奶奶,简直是她要甚么就给她甚么.一次,奶奶送给小姑娘一顶用丝绒做的小红帽,戴在她的头上正好合适.从此,姑娘再也不愿意戴任何别的帽子,於是大家便叫她「小红帽」.

一天,妈妈对小红帽说：「来,小红帽,这里有一块蛋糕和一瓶葡萄酒,快给奶奶送去,奶奶生病了,身子很虚弱,吃了这些就会好一些的.趁着现在天还没有热,赶紧动身吧.在路上要好好走,不要跑,也不要离开大路,否则你会摔跤的,那样奶奶就甚么也吃不上了.到奶奶家的时候,别忘了说’早上好’,也不要一进屋就东瞧西瞅.」

「我会小心的.」小红帽对妈妈说,并且还和妈妈拉手作保证.

奶奶住在村子外面的森林里,离小红帽家有很长一段路.小红帽刚走进森林就碰到了一条狼.小红帽不知道狼是坏家伙,所以一点也不怕它.

小红帽,」狼说.

狼先生.」

「小红帽,这么早要到哪里去呀?」

「我要到奶奶家去.」

「你那围裙下面有甚么呀?」

「蛋糕和葡萄酒.昨天我们家烤了一些蛋糕,可怜的奶奶生了病,要吃一些好东西才能恢复过来.」

「你奶奶住在哪里呀,小红帽?」

「进了林子还有一段路呢.她的房子就在三棵大橡树下,低处围着核桃树篱笆.你一定知道的.」小红帽说.

狼在心中盘算着：「这小东西细皮嫩肉的,味道肯定比那老太婆要好.我要讲究一下策略,让她俩都逃不出我的手心.」於是它陪着小红帽走了一会儿,然后说：「小红帽,你看周围这些花多么美丽啊!干吗不回头看一看呢?还有这些小鸟,它们唱得多么动听啊!你大概根本没有听到吧?林子里的一切多么美好啊,而你却只管往前走,就像是去上学一样.」

小红帽抬起头来,看到阳光在树木间来回跳荡,美丽的鲜花在四周开放,便想：「也许我该摘一把鲜花给奶奶,让她高兴高兴.现在天色还早,我不会去迟的.」她於是离开大路,走进林子去采花.她每采下一朵花,总觉得前面还有更美丽的花朵,便又向前走去,结果一直走到了林子深处.

就在此时,狼却直接跑到奶奶家,敲了敲门.

「是谁呀?」

「是小红帽.」狼回答,「我给你送蛋糕和葡萄酒来了.快开门哪.」

「你拉一下门栓就行了,」奶奶大声说,「我身上没有力气,起不来.」

狼刚拉起门栓,那门就开了.

## 0.Load LLM
1. vllm.sh 起一个LLM服务
2. 定义接口调用

In [8]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    api_key="YOUR API KEY",
    base_url="http://localhost:8080/v1",
    temperature=0,
    model_name='Qwen2.5-3B-Instruct' 
)

llm.invoke("hello.")

AIMessage(content='Hello! How can I assist you today? Feel free to ask any questions or let me know if you need information on a particular topic.', response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 31, 'total_tokens': 60}, 'model_name': 'Qwen2.5-3B-Instruct', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-cb99e586-8e3a-45e7-aeaf-673fbc6f9add-0')

## 1.文本总结

1. chain-文本总结任务(load_summarize_chain https://python.langchain.ac.cn/v0.2/docs/tutorials/summarization/)
2. load_summarize_chain的三种模式

### 1.1 stuff模式

In [12]:
from langchain.prompts import PromptTemplate

prompt_template = """Write a concise summary of the following:
{text}
"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])

# summarize_chain (verbose 参数为 True，会打印全部的执行详情)
chain = load_summarize_chain(llm, chain_type="stuff", prompt=PROMPT, verbose=True)
chain.run(lg_doc)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:
从前有个可爱的小姑娘,谁见了都喜欢,但最喜欢她的是她的奶奶,简直是她要甚么就给她甚么.一次,奶奶送给小姑娘一顶用丝绒做的小红帽,戴在她的头上正好合适.从此,姑娘再也不愿意戴任何别的帽子,於是大家便叫她「小红帽」.

一天,妈妈对小红帽说：「来,小红帽,这里有一块蛋糕和一瓶葡萄酒,快给奶奶送去,奶奶生病了,身子很虚弱,吃了这些就会好一些的.趁着现在天还没有热,赶紧动身吧.在路上要好好走,不要跑,也不要离开大路,否则你会摔跤的,那样奶奶就甚么也吃不上了.到奶奶家的时候,别忘了说’早上好’,也不要一进屋就东瞧西瞅.」

「我会小心的.」小红帽对妈妈说,并且还和妈妈拉手作保证.

奶奶住在村子外面的森林里,离小红帽家有很长一段路.小红帽刚走进森林就碰到了一条狼.小红帽不知道狼是坏家伙,所以一点也不怕它.

小红帽,」狼说.

狼先生.」

「小红帽,这么早要到哪里去呀?」

「我要到奶奶家去.」

「你那围裙下面有甚么呀?」

「蛋糕和葡萄酒.昨天我们家烤了一些蛋糕,可怜的奶奶生了病,要吃一些好东西才能恢复过来.」

「你奶奶住在哪里呀,小红帽?」

「进了林子还有一段路呢.她的房子就在三棵大橡树下,低处围着核桃树篱笆.你一定知道的.」小红帽说.

狼在心中盘算着：「这小东西细皮嫩肉的,味道肯定比那老太婆要好.我要讲究一下策略,让她俩都逃不出我的手心.」於是它陪着小红帽走了一会儿,然后说：「小红帽,你看周围这些花多么美丽啊!干吗不回头看一看呢?还有这些小鸟,它们唱得多么动听啊!你大概根本没有听到吧?林子里的一切多么美好啊,而你却只管往前走,就像是去上学一样.」

小红帽抬起头来,看到阳光在树木间来回跳荡,美丽的鲜花在四周开放,便想：「也许我该摘一把鲜花给奶奶,让她高兴高兴.现在天色还早,我不会去迟的.」她於是离开大路,走进林子去采花.她每采下一朵花,总觉得前面还有更美丽的花朵,便又向前走去,结果一直走到了林子深处.

就在此时,狼却直接跑到奶奶家,敲了敲门.

「是谁呀?」

「是小

'从前有一个可爱的小姑娘，大家都喜欢她，但她最喜欢的是她的奶奶。奶奶总是满足她的所有要求，比如给她做了一顶用丝绒做的小红帽。从此，小姑娘再也不愿意戴其他帽子，大家便叫她“小红帽”。\n\n一天，妈妈告诉小红帽，奶奶生病了，需要一些蛋糕和葡萄酒来恢复健康。妈妈嘱咐小红帽要小心，不要走远，不要离开大路，否则会摔跤。小红帽答应了妈妈，并且和妈妈拉手保证会小心。\n\n奶奶住在村子外的森林里，离小红帽家很远。小红帽刚走进森林就遇到了一只狼。小红帽并不害怕，她和狼交谈，但狼却在心中盘算着如何利用小红帽去接近奶奶。最终，狼骗过小红帽，潜入奶奶的家，吞下了奶奶。小红帽发现奶奶后，发现奶奶被狼吞了进去，狼则穿上奶奶的衣服躺在床上。小红帽意识到情况的严重性，她大声叫奶奶，但没有得到回应。她看到奶奶躺在床上，帽子拉得低低的，样子非常奇怪。小红帽意识到奶奶被狼吃掉了，于是她开始寻找奶奶。她发现狼在狼肚子里，猎人正好路过，发现了狼。猎人剪开了狼的肚子，救出了小红帽和奶奶。\n\n从此，小红帽再也不独自离开大路，跑进森林了。人们还说，小红帽后来又送蛋糕给奶奶，但这次她提高了警惕，没有被狼骗。她告诉奶奶遇到了狼，奶奶则让小红帽去拿一个装满水的桶，狼闻到香肠的气味，伸长了脖子，滑进了装满水的桶里，淹死了。\n\n从此，小红帽和奶奶过上了幸福的生活，而小红帽也学会了更加警惕和小心。'

### 1.2 Map Reduce模式

In [24]:
prompt_template = """Write a concise summary of the following:
{text}
"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True)
chain.run(lg_doc)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"从前有个可爱的小姑娘,谁见了都喜欢,但最喜欢她的是她的奶奶,简直是她要甚么就给她甚么.一次,奶奶送给小姑娘一顶用丝绒做的小红帽,戴在她的头上正好合适.从此,姑娘再也不愿意戴任何别的帽子,於是大家便叫她「小红帽」.

一天,妈妈对小红帽说：「来,小红帽,这里有一块蛋糕和一瓶葡萄酒,快给奶奶送去,奶奶生病了,身子很虚弱,吃了这些就会好一些的.趁着现在天还没有热,赶紧动身吧.在路上要好好走,不要跑,也不要离开大路,否则你会摔跤的,那样奶奶就甚么也吃不上了.到奶奶家的时候,别忘了说’早上好’,也不要一进屋就东瞧西瞅.」

「我会小心的.」小红帽对妈妈说,并且还和妈妈拉手作保证.

奶奶住在村子外面的森林里,离小红帽家有很长一段路.小红帽刚走进森林就碰到了一条狼.小红帽不知道狼是坏家伙,所以一点也不怕它.

小红帽,」狼说.

狼先生.」

「小红帽,这么早要到哪里去呀?」

「我要到奶奶家去.」

「你那围裙下面有甚么呀?」

「蛋糕和葡萄酒.昨天我们家烤了一些蛋糕,可怜的奶奶生了病,要吃一些好东西才能恢复过来.」

「你奶奶住在哪里呀,小红帽?」

「进了林子还有一段路呢.她的房子就在三棵大橡树下,低处围着核桃树篱笆.你一定知道的.」小红帽说.

狼在心中盘算着：「这小东西细皮嫩肉的,味道肯定比那老太婆要好.我要讲究一下策略,让她俩都逃不出我的手心.」於是它陪着小红帽走了一会儿,然后说：「小红帽,你看周围这些花多么美丽啊!干吗不回头看一看呢?还有这些小鸟,它们唱得多么动听啊!你大概根本没有听到吧?林子里的一切多么美好啊,而你却只管往前走,就像是去上学一样.」

小红帽抬起头来,看到阳光在树木间来回跳荡,美丽的鲜花在四周开放,便想：「也许我该摘一把鲜花给奶奶,让她高兴高兴.现在天色还早,我不会去迟的.」她於是离开大路,走进林子去采花.她每采下一朵花,总觉得前面还有更美丽的花朵,便又向前走去,结果一直走到了林子深处.

就在此时,狼却直接跑到奶奶家,敲了敲门.

「是谁呀


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"从前有一个可爱的小姑娘，因为奶奶的宠爱而被称为“小红帽”。一天，妈妈让她给生病的奶奶送去蛋糕和葡萄酒。在前往奶奶家的路上，小红帽遇到了一只狼。狼试图引诱小红帽偏离大路，但小红帽保持警惕，最终到达奶奶家。狼趁机将奶奶吞下，伪装成奶奶躺在床上。小红帽发现奶奶异常后，意识到危险，勇敢地揭露了狼的真面目。猎人到来后，发现狼是伪装的奶奶，将其打死。奶奶获救，小红帽也学会了独自面对危险。之后，小红帽又送蛋糕给奶奶，但这次她提高了警惕，成功避免了狼的再次袭击。从此，小红帽学会了独自面对危险，而狼则被猎人杀死。"


CONCISE SUMMARY:

> Finished chain.

> Finished chain.


'从前有一个可爱的小姑娘，因奶奶宠爱而被称为“小红帽”。一天，她被妈妈派去给生病的奶奶送蛋糕和葡萄酒。途中遇到一只狼，试图引诱她偏离大路，但小红帽保持警惕，安全抵达奶奶家。狼趁机吞下奶奶，伪装成奶奶躺在床上。小红帽察觉异常后揭露狼的真面目，猎人到来后将其打死。奶奶获救，小红帽也学会了独自应对危险。之后，她提高警惕，再次送蛋糕给奶奶，成功避免了狼的再次袭击。从此，小红帽学会了独自面对危险，而狼则被猎人杀死。'

In [20]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 400,
    chunk_overlap = 0
)

lg_docs_split = text_splitter.split_documents(lg_doc)
lg_docs_split

[Document(page_content='从前有个可爱的小姑娘,谁见了都喜欢,但最喜欢她的是她的奶奶,简直是她要甚么就给她甚么.一次,奶奶送给小姑娘一顶用丝绒做的小红帽,戴在她的头上正好合适.从此,姑娘再也不愿意戴任何别的帽子,於是大家便叫她「小红帽」.\n\n一天,妈妈对小红帽说：「来,小红帽,这里有一块蛋糕和一瓶葡萄酒,快给奶奶送去,奶奶生病了,身子很虚弱,吃了这些就会好一些的.趁着现在天还没有热,赶紧动身吧.在路上要好好走,不要跑,也不要离开大路,否则你会摔跤的,那样奶奶就甚么也吃不上了.到奶奶家的时候,别忘了说’早上好’,也不要一进屋就东瞧西瞅.」\n\n「我会小心的.」小红帽对妈妈说,并且还和妈妈拉手作保证.\n\n奶奶住在村子外面的森林里,离小红帽家有很长一段路.小红帽刚走进森林就碰到了一条狼.小红帽不知道狼是坏家伙,所以一点也不怕它.\n\n小红帽,」狼说.\n\n狼先生.」\n\n「小红帽,这么早要到哪里去呀?」\n\n「我要到奶奶家去.」', metadata={'source': '../data/小红帽.txt'}),
 Document(page_content='「你那围裙下面有甚么呀?」\n\n「蛋糕和葡萄酒.昨天我们家烤了一些蛋糕,可怜的奶奶生了病,要吃一些好东西才能恢复过来.」\n\n「你奶奶住在哪里呀,小红帽?」\n\n「进了林子还有一段路呢.她的房子就在三棵大橡树下,低处围着核桃树篱笆.你一定知道的.」小红帽说.\n\n狼在心中盘算着：「这小东西细皮嫩肉的,味道肯定比那老太婆要好.我要讲究一下策略,让她俩都逃不出我的手心.」於是它陪着小红帽走了一会儿,然后说：「小红帽,你看周围这些花多么美丽啊!干吗不回头看一看呢?还有这些小鸟,它们唱得多么动听啊!你大概根本没有听到吧?林子里的一切多么美好啊,而你却只管往前走,就像是去上学一样.」', metadata={'source': '../data/小红帽.txt'}),
 Document(page_content='小红帽抬起头来,看到阳光在树木间来回跳荡,美丽的鲜花在四周开放,便想：「也许我该摘一把鲜花给奶奶,让她高兴高兴.现在天色还早,我不会去迟的.」她於是离开大路,走进林子去采花.她每采下一朵花,总觉得前面还有更美丽的花朵,便又向前走去,结果一直走到了林子深处.\n\n就

In [22]:
doc_summary(lg_docs_split)

You have 8 document(s)
You have roughly 8 words in your docs
Preview: 
从前有个可爱的小姑娘,谁见了都喜欢,但最喜欢她的是她的奶奶,简直是她要甚么就给她甚么.一次,奶奶送给小姑娘一顶用丝绒做的小红帽,戴在她的头上正好合适.从此,姑娘再也不愿意戴任何别的帽子,於是大家便叫她「小红帽」.

一天,妈妈对小红帽说：「来,小红帽,这里有一块蛋糕和一瓶葡萄酒,快给奶奶送去,奶奶生病了,身子很虚弱,吃了这些就会好一些的.趁着现在天还没有热,赶紧动身吧.在路上要好好走,不要跑,也不要离开大路,否则你会摔跤的,那样奶奶就甚么也吃不上了.到奶奶家的时候,别忘了说’早上好’,也不要一进屋就东瞧西瞅.」

「我会小心的.」小红帽对妈妈说,并且还和妈妈拉手作保证.

奶奶住在村子外面的森林里,离小红帽家有很长一段路.小红帽刚走进森林就碰到了一条狼.小红帽不知道狼是坏家伙,所以一点也不怕它.

小红帽,」狼说.

狼先生.」

「小红帽,这么早要到哪里去呀?」

「我要到奶奶家去.」


In [25]:
chain.run(lg_docs_split)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"从前有个可爱的小姑娘,谁见了都喜欢,但最喜欢她的是她的奶奶,简直是她要甚么就给她甚么.一次,奶奶送给小姑娘一顶用丝绒做的小红帽,戴在她的头上正好合适.从此,姑娘再也不愿意戴任何别的帽子,於是大家便叫她「小红帽」.

一天,妈妈对小红帽说：「来,小红帽,这里有一块蛋糕和一瓶葡萄酒,快给奶奶送去,奶奶生病了,身子很虚弱,吃了这些就会好一些的.趁着现在天还没有热,赶紧动身吧.在路上要好好走,不要跑,也不要离开大路,否则你会摔跤的,那样奶奶就甚么也吃不上了.到奶奶家的时候,别忘了说’早上好’,也不要一进屋就东瞧西瞅.」

「我会小心的.」小红帽对妈妈说,并且还和妈妈拉手作保证.

奶奶住在村子外面的森林里,离小红帽家有很长一段路.小红帽刚走进森林就碰到了一条狼.小红帽不知道狼是坏家伙,所以一点也不怕它.

小红帽,」狼说.

狼先生.」

「小红帽,这么早要到哪里去呀?」

「我要到奶奶家去.」"


CONCISE SUMMARY:
Prompt after formatting:
Write a concise summary of the following:


"「你那围裙下面有甚么呀?」

「蛋糕和葡萄酒.昨天我们家烤了一些蛋糕,可怜的奶奶生了病,要吃一些好东西才能恢复过来.」

「你奶奶住在哪里呀,小红帽?」

「进了林子还有一段路呢.她的房子就在三棵大橡树下,低处围着核桃树篱笆.你一定知道的.」小红帽说.

狼在心中盘算着：「这小东西细皮嫩肉的,味道肯定比那老太婆要好.我要讲究一下策略,让她俩都逃不出我的手心.」於是它陪着小红帽走了一会儿,然后说：「小红帽,你看周围这些花多么美丽啊!干吗不回头看一看呢?还有这些小鸟,它们唱得多么动听啊!你大概根本没有听到吧?林子里的一切多么美好啊,而你却只管往前走,就像是去上学一样.」"


CONCISE SUMMARY:
Prompt after formatting:
Write a concise summary


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"从前有个可爱的小姑娘，因为奶奶的宠爱而被称为“小红帽”。一天，妈妈让小红帽给生病的奶奶送去蛋糕和葡萄酒。小红帽出发时妈妈叮嘱她要小心，不要走偏路，不要东张西望。小红帽走进森林后遇到了一条狼，误以为狼是她的朋友，于是和狼对话。狼趁机欺骗小红帽，让她独自前往奶奶家，企图吃掉她。

狼在得知小红帽家烤了蛋糕和葡萄酒，奶奶生病需要这些食物后，心中盘算着如何利用这些信息。它陪着小红帽走了一段时间，然后引导她注意周围的美丽景色，试图分散她的注意力，以便在她和奶奶面前设下圈套。

小红帽决定去采花送给奶奶，但因贪玩一直走到林子深处。这时，狼已经潜入奶奶家，骗奶奶说是小红帽送来了蛋糕和葡萄酒。奶奶因虚弱无法开门，狼趁机进入并吞掉了奶奶。小红帽在采花过程中忘了奶奶，直到采花完毕才意识到，匆忙赶往奶奶家，但为时已晚。

小红帽发现奶奶家的屋门敞开着，走进去后感到奇怪和害怕。她先是问候奶奶，但没有得到回应。随后，她看到奶奶躺在床上，帽子拉得很低，遮住了脸。小红帽询问奶奶的耳朵、眼睛和手为什么这么大，奶奶解释说是为了更好地听她说话、看她和更好地抱着她。最后，小红帽问奶奶嘴巴为什么大，奶奶开玩笑说可以一口把她吃掉。

狼吞下了小红帽和她的奶奶后，满足地睡着了。一位猎人听到奶奶的鼾声，决定进去查看。进入屋内后，猎人发现床上躺着的竟是狼。猎人犹豫了一下，考虑到狼可能吞下了奶奶，便没有开枪，而是用剪刀剪开了狼的肚子。发现红色的小红帽后，猎人又剪开了狼的肚子，奶奶也从狼的肚子中出来，但有些喘不过气。小红帽赶紧搬来石头塞进狼的肚子，狼醒来后因石头过重而摔倒死亡。

三个人都很高兴：猎人剥下狼皮回家，奶奶吃了小红帽带来的蛋糕和葡萄酒后精神好转。然而，小红帽独自进入森林的经历让她意识到，如果妈妈不允许，她将永远不再独自一人进入森林。

小红帽再次给奶奶送蛋糕时，提高了警惕，没有理会路上的狼。狼试图骗她们开门，但她们没有开门。狼在房子周围转了几圈后，跳上屋顶打算在傍晚时偷袭小红帽。奶奶想起屋子前有一个装满水的大石头槽子，让小红帽去取水。小红帽提了很多水，装满石头槽

'从前有个可爱的小姑娘，因奶奶的宠爱被称为“小红帽”。一天，妈妈让她给生病的奶奶送去蛋糕和葡萄酒，嘱咐她要小心。小红帽走进森林后遇到了一条狼，误以为狼是朋友，与狼对话。狼趁机欺骗小红帽独自前往奶奶家，企图吃掉她。狼在得知奶奶家有蛋糕和葡萄酒后，潜入奶奶家，骗奶奶说是小红帽送来的，奶奶因虚弱无法开门，狼趁机进入并吞掉了奶奶。小红帽发现奶奶家的屋门敞开着，走进去后发现奶奶躺在床上，帽子拉得很低，遮住了脸。奶奶解释说是为了更好地听她说话、看她和更好地抱着她。小红帽问奶奶嘴巴为什么大，奶奶开玩笑说可以一口把她吃掉。狼吞下了小红帽和奶奶后，猎人发现狼并将其杀死。奶奶从狼的肚子中出来，但有些喘不过气。小红帽搬来石头塞进狼的肚子，狼因石头过重而摔倒死亡。小红帽再次给奶奶送蛋糕时，妈妈让她提高警惕，最终平安回家。'

### 1.3 Refine模式

In [26]:
prompt_template = """Write a concise summary of the following:
{text}
"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
chain = load_summarize_chain(llm, chain_type="refine", prompt=PROMPT, verbose=True)
chain.run(lg_docs_split)


ValidationError: 1 validation error for RefineDocumentsChain
prompt
  extra fields not permitted (type=value_error.extra)

In [27]:
prompt_template = """写出以下内容的简洁摘要:
{text}
"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
refine_template = (
    "你是一个摘要助手， 你的任务是进行最终摘要\n"
    "提供了一定程度的现有摘要: {existing_answer}\n"
    "下面有更多背景信息进行补充.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "鉴于新的背景，进一步完善原始摘要，如果上下文对于总结摘要没有用，则返回原始摘要。"
)
refine_prompt = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)
chain = load_summarize_chain(llm, chain_type="refine", return_intermediate_steps=True, question_prompt=PROMPT, refine_prompt=refine_prompt)
chain({"input_documents": lg_docs_split}, return_only_outputs=True)


{'intermediate_steps': ['从前有个可爱的小姑娘，因为奶奶的宠爱而被称为“小红帽”。一天，妈妈让她给生病的奶奶送去蛋糕和葡萄酒。途中，小红帽遇到了一条狼，误以为狼是她的朋友。狼假装是奶奶，骗小红帽带路。最终，小红帽识破了狼的诡计，保住了奶奶的食物，避免了一场危机。从此，小红帽学会了警惕和智慧。',
  '完善后的摘要如下：\n\n从前有个可爱的小姑娘，因为奶奶的宠爱而被称为“小红帽”。一天，妈妈让她给生病的奶奶送去蛋糕和葡萄酒。途中，小红帽遇到了一条狼，误以为狼是她的朋友。狼假装是奶奶，骗小红帽带路。在狼的引诱下，小红帽开始分心，对周围美丽的景色产生了兴趣，忘记了保护自己。最终，小红帽识破了狼的诡计，保住了奶奶的食物，避免了一场危机。这次经历让小红帽学会了警惕和智慧，也让她更加了解了保护自己和信任的重要性。',
  '根据提供的背景信息，完善后的摘要如下：\n\n从前有个可爱的小姑娘，因为奶奶的宠爱而被称为“小红帽”。一天，妈妈让她给生病的奶奶送去蛋糕和葡萄酒。途中，小红帽遇到了一条狼，误以为狼是她的朋友。狼假装是奶奶，骗小红帽带路。在狼的引诱下，小红帽开始分心，对周围美丽的景色产生了兴趣，忘记了保护自己。小红帽在林子深处采花时，狼趁机潜入奶奶家，骗过奶奶，将她吞进肚子。狼穿上奶奶的衣服，戴上她的帽子，躺在床上，还拉上了帘子。小红帽直到采了许多花，才想起奶奶，重新上路去奶奶家。最终，小红帽识破了狼的诡计，保住了奶奶的食物，避免了一场危机。这次经历让小红帽学会了警惕和智慧，也让她更加了解了保护自己和信任的重要性。',
  '根据提供的背景信息，完善后的摘要如下：\n\n从前有个可爱的小姑娘，因为奶奶的宠爱而被称为“小红帽”。一天，妈妈让她给生病的奶奶送去蛋糕和葡萄酒。途中，小红帽遇到了一条狼，误以为狼是她的朋友。狼假装是奶奶，骗小红帽带路。在狼的引诱下，小红帽开始分心，对周围美丽的景色产生了兴趣，忘记了保护自己。小红帽在林子深处采花时，狼趁机潜入奶奶家，骗过奶奶，将她吞进肚子。狼穿上奶奶的衣服，戴上她的帽子，躺在床上，还拉上了帘子。小红帽直到采了许多花，才想起奶奶，重新上路去奶奶家。在接近奶奶家时，她发现屋门敞开着，感到很奇怪。她一走进屋子就有一种异样的感觉，心中便想：“天哪!平常我那么喜欢来奶奶家，今天怎么这样害怕?”她大声叫道：“早上好!”，

### Map Re-Rank
- QA任务

In [ ]:
chain = load_qa_chain(llm, chain_type="map_rerank", verbose=True, return_intermediate_steps=True)

In [29]:
query = "小红帽中的狼是什么性别？"

result = chain({"input_documents": lg_docs_split, "question": query}, return_only_outputs=True)



> Entering new MapRerankDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

In addition to giving an answer, also return a score of how fully it answered the user's question. This should be in the following format:

Question: [question here]
Helpful Answer: [answer here]
Score: [score between 0 and 100]

How to determine the score:
- Higher is a better answer
- Better responds fully to the asked question, with sufficient level of detail
- If you do not know the answer based on the context, that should be a score of 0
- Don't be overconfident!

Example #1

Context:
---------
Apples are red
---------
Question: what color are apples?
Helpful Answer: red
Score: 100

Example #2

Context:
---------
it was night and the witness forgot his glasses. he was not sure if it was a sports car or an suv
--

In [30]:
result['output_text']

'这个故事中并没有明确提到狼的性别。'

In [31]:
result['intermediate_steps']

[{'answer': '这个故事中并没有明确提到狼的性别。', 'score': '0'},
 {'answer': '这个段落中并没有明确提到狼的性别。', 'score': '0'},
 {'answer': '这个故事中并没有明确提到狼的性别。根据描述，狼是主动接近并攻击小红帽的奶奶的，但没有直接说明狼的性别。',
  'score': '0'},
 {'answer': '这个段落描述的是小红帽的故事，其中的狼并没有明确指出性别。因此，根据提供的信息，无法确定狼的性别。', 'score': '0'},
 {'answer': '这个故事中并没有明确提到狼的性别。', 'score': '0'},
 {'answer': '这个段落并没有提到狼的性别。', 'score': '0'},
 {'answer': '这个文本描述的是一个关于小红帽的故事，但并没有明确提到狼的性别。文本中只描述了狼的行为和特征，但没有直接说明狼的性别。',
  'score': '0'},
 {'answer': 'This document does not answer the question', 'score': '0'}]

#### 自定义实现


In [39]:
from langchain.chains import LLMChain, MapRerankDocumentsChain
from langchain.output_parsers.regex import RegexParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI

document_variable_name = "context"
# 其中例子
prompt_template = (
    "小红帽中的狼是什么性别？"
    "Output both your answer and a score (1-10) of how confident "
    "you are in the format: <Answer>\nScore: <Score>.\n\n"
    "Provide no other commentary.\n\n"
    "Context: {context}"
)
output_parser = RegexParser(
    regex=r"(.*?)\nScore: (.*)",
    output_keys=["answer", "score"],
)
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context"],
    output_parser=output_parser,
)
llm_chain = LLMChain(llm=llm, prompt=prompt)
chain = MapRerankDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name=document_variable_name,
    rank_key="score",
    answer_key="answer",
    return_intermediate_steps=True
)

result = chain({"input_documents": lg_docs_split, "question": query}, return_only_outputs=True)

In [40]:
result['output_text']

'狼是雄性。'

In [41]:
result['intermediate_steps']

[{'answer': '狼是雄性。', 'score': '10'},
 {'answer': '狼是雄性。', 'score': '10'},
 {'answer': '狼是雄性。', 'score': '10'},
 {'answer': '狼在《小红帽》的故事中是男性角色。  ', 'score': '10'},
 {'answer': '狼是雄性。', 'score': '10'},
 {'answer': '狼是雄性。', 'score': '9'},
 {'answer': '狼是雄性。', 'score': '10'},
 {'answer': 'Answer: Wolf', 'score': '1'}]